# SparkSQL and MLib Assignment

## Part A: SparkSQL

#### Course: Big Data Systems and Architectures           Master: Business Analytics - Full Time
#### Student Name: Lakkas - Pyknis Evangelos              Student ID:f2822306     email: eva.lakkaspyknis@aueb.gr  

In [120]:
from pyspark.sql import SparkSession

In [121]:
from pyspark import SparkContext, SparkConf

In [122]:
import pandas as pd

In [123]:
spark = SparkSession.builder \
    .appName("IMDB Analysis") \
    .getOrCreate()


In [124]:
imdb_df = spark.read.json("movie.json")

In [125]:
imdb_df.createOrReplaceTempView("imdb_data")

In [126]:
pandas_imdb = imdb_df.toPandas()

In [127]:
pandas_imdb.head(3)

,_corrupt_record,actors,countries,description,directors,genre,imdb_url,img_url,languages,metascore,rating,runtime,tagline,title,users_rating,votes,year
0,[,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,"[Timothée Chalamet, Elle Fanning, Liev Schreib...",[USA],A young couple arrives in New York for a weeke...,[Woody Allen],"[Comedy, Romance]",https://www.imdb.com/title/tt7139936/,https://m.media-amazon.com/images/M/MV5BODAwZD...,[English],44,PG-13,92 min,None,A Rainy Day in New York,6.6,"21,903",2019
2,None,"[Emilia Clarke, Hadley Fraser, Sylvia Panacion...",[USA],"Creepy, terrifying chapters from our book of h...","[Michael Escobedo, Kelly Hallmark, 4 more cred...","[Horror, Thriller]",https://www.imdb.com/title/tt12384178/,https://m.media-amazon.com/images/M/MV5BMDI5ZD...,[English],None,18,91 min,"When it comes to murder, they wrote the book",Murder Manual,2.4,192,2020


## Task A2

*showing the movie with the lowest rating that starts with the letter V and has at least 200,000 votes*

In [128]:
query1 = """
    SELECT title, year, users_rating
    FROM imdb_data
    WHERE title LIKE 'V%' AND votes >= 200000
    ORDER BY users_rating
    LIMIT 1"""

In [129]:
v_movie = spark.sql(query1)

In [130]:
v_movie.show()

+-----+----+------------+
|title|year|users_rating|
+-----+----+------------+
+-----+----+------------+



Unfortunately there is no movie in the data that starts from V and has at least 200000 votes however if I change
the votes criterion the code works fine

In [131]:
query11 = """
    SELECT title, year, users_rating
    FROM imdb_data
    WHERE title LIKE 'V%' AND votes >= 200
    ORDER BY users_rating
    LIMIT 1"""

In [132]:
v_movie_test = spark.sql(query11)

In [133]:
v_movie_test.show()

+----------+----+------------+
|     title|year|users_rating|
+----------+----+------------+
|Voyeur.com|2000|         1.8|
+----------+----+------------+



*showing the average metascore and average rating for the movies that start with the letter A and have at least 200,000 votes*

In [137]:
query2 = """ SELECT AVG(users_rating) AS avg_users_rating, AVG(metascore) AS avg_metascore
    FROM imdb_data
    WHERE title LIKE 'A%' AND votes >= 200000 AND users_rating IS NOT NULL AND metascore IS NOT NULL
"""

In [138]:
a_movie = spark.sql(query2)

In [139]:
a_movie.show()

+----------------+-------------+
|avg_users_rating|avg_metascore|
+----------------+-------------+
|            null|         null|
+----------------+-------------+



In [79]:
spark.sql("SELECT COUNT(*) FROM imdb_data WHERE users_rating IS NULL OR metascore IS NULL").show()

+--------+
|count(1)|
+--------+
|   52449|
+--------+



Question 3
* *Title, year, and votes of the movie with the most votes starting with the letter N*

I created a query that selects the wanted columns and filters the data for movies starting with the letter N it orders them based on how many votes they have descending order and displays only the movie on the top of this sorted list

In [143]:
query3 = """
    SELECT title, year, votes
    FROM imdb_data
    WHERE title LIKE 'N%'
    ORDER BY votes
    LIMIT 1"""

In [144]:
n_movies = spark.sql(query3)

In [145]:
n_movies.show()

+------------+----+-----+
|       title|year|votes|
+------------+----+-----+
|Night People|1954|1,004|
+------------+----+-----+



Question 4

* *Maximum and Minimum users_rating score for all Bruce Willis movies*

Here I created a query that selects the minimum and maximum value of the users rating column only for the movie titles in which Bruce Willis has participated this happens with the execution of an embded query and the use of the array_contains function which checks if the actors column of the JSON dataset (which is a list of names/strings) contains Bruce Willis

In [146]:
query4 = """
    SELECT MAX(users_rating) AS max_users_rating, MIN(users_rating) AS min_users_rating
    FROM imdb_data
    WHERE title IN (
        SELECT title
        FROM imdb_data
        WHERE array_contains(actors, 'Bruce Willis')
    )
"""

In [147]:
bw_movies = spark.sql(query4)

The results of the query are displayed below

In [148]:
bw_movies.show()

+----------------+----------------+
|max_users_rating|min_users_rating|
+----------------+----------------+
|             8.9|             3.6|
+----------------+----------------+



In [86]:
spark.stop()